In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models as torchvision_models
from torch.utils.data import DataLoader, random_split
from tqdm.auto import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from torchvision.models import resnet101, ResNet101_Weights
from torchvision.models import resnet50, ResNet50_Weights


In [ ]:
pip install timm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet50, resnet101
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from torchvision.models.resnet import resnet101, ResNet101_Weights
from torchvision.datasets import ImageFolder

In [ ]:
# Define SelfAttention and Custom models
class SelfAttention(nn.Module):
    def __init__(self, embed_size):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.query = nn.Linear(embed_size, embed_size)
        self.key = nn.Linear(embed_size, embed_size)
        self.value = nn.Linear(embed_size, embed_size)

    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        attention_weights = torch.softmax(torch.bmm(Q, K.permute(0, 2, 1)) / (self.embed_size ** 0.5), dim=-1)
        out = torch.bmm(attention_weights, V)
        return out.squeeze(1)

class CustomResNet50(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet50, self).__init__()
        original_model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.features = nn.Sequential(*list(original_model.children())[:-2])
        self.attention = SelfAttention(embed_size=2048)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.attention(x.unsqueeze(1))
        x = self.classifier(x)
        return x

class CustomResNet101(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet101, self).__init__()
        original_model = resnet101(weights=ResNet101_Weights.DEFAULT)
        self.features = nn.Sequential(*list(original_model.children())[:-2])
        self.attention = SelfAttention(embed_size=2048)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.attention(x.unsqueeze(1))
        x = self.classifier(x)
        return x

In [ ]:
# Correct data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Lambda(lambda x: x.convert('RGB')),
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Lambda(lambda x: x.convert('RGB')),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



# Load image data
train_dir = '/kaggle/input/ham10000-data/HAM10000_DATA/train_dir'
# val_dir = '/kaggle/input/ham10000-data/HAM10000_DATA/test_dir'

#Use train_dir to create both train and val datasets since dataset only has train and test data
full_dataset = ImageFolder(train_dir,transform=None)
train_ratio = 0.8
train_size = int(train_ratio * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset,val_dataset = random_split(full_dataset,[train_size,val_size])

# Apply different transforms to train and val datasets by overriding the transform attribute
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

# image_datasets = {
#     'train': ImageFolder(train_dir, transform=data_transforms['train']),
#     'val': ImageFolder(val_dir, transform=data_transforms['val']),
# }

# Create DataLoader objects
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=4, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=4, shuffle=False),
}

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 7
criterion = nn.CrossEntropyLoss()

# Initialize models and optimizers
custom_resnet50 = CustomResNet50(num_classes).to(device)
custom_resnet101 = CustomResNet101(num_classes).to(device)
models = {
    'CustomResNet50': {'model': custom_resnet50, 'optimizer': optim.Adam(custom_resnet50.parameters(), lr=0.0001)},
    'CustomResNet101': {'model': custom_resnet101, 'optimizer': optim.Adam(custom_resnet101.parameters(), lr=0.0001)}
}

In [ ]:

# Training and evaluation functions as previously provided
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(dataloader, desc="Training", leave=False):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating", leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc

def compete_until_equilibrium(models, dataloaders, criterion, device, num_epochs=25, threshold=0.001):
    std_devs = []
    best_std = float("inf")
    performance_metrics = {name: {'last_val_loss': float('inf'), 'best_val_acc': 0} for name in models.keys()}

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}\n{'=' * 60}")
        model_metrics = {}

        for name, model_info in models.items():
            model = model_info['model']
            optimizer = model_info['optimizer']

            train_loss, train_acc = train_one_epoch(model, dataloaders['train'], criterion, optimizer, device)
            val_loss, val_acc = evaluate_model(model, dataloaders['val'], criterion, device)
            print(f"{name} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

            model_metrics[name] = {'val_loss': val_loss, 'val_acc': val_acc}

        losses_list = [metrics['val_loss'] for metrics in model_metrics.values()]
        std_dev = np.std(losses_list)
        std_devs.append(std_dev)
        print(f"Standard Deviation of Validation Losses: {std_dev:.4f}")

        if std_dev < best_std:
            best_std = std_dev
            for name, model_info in models.items():
                if std_dev < performance_metrics[name]['last_val_loss'] or model_metrics[name]['val_acc'] > performance_metrics[name]['best_val_acc']:
                    torch.save(model_info['model'].state_dict(), f"/kaggle/working/{name}.pth")
                    print(f'===> {name} model saved based on new std or val acc improvement')
                    performance_metrics[name]['best_val_acc'] = model_metrics[name]['val_acc']

        sorted_models = sorted(model_metrics.keys(), key=lambda x: (model_metrics[x]['val_loss'], -model_metrics[x]['val_acc']))
        for rank, name in enumerate(sorted_models):
            improvement = performance_metrics[name]['last_val_loss'] - model_metrics[name]['val_loss']
            adjust_factor = max(0.9, 1 - 0.05 * improvement)
            new_lr = models[name]['optimizer'].param_groups[0]['lr'] * adjust_factor
            new_lr = max(new_lr, 1e-6)
            models[name]['optimizer'].param_groups[0]['lr'] = new_lr
            performance_metrics[name]['last_val_loss'] = model_metrics[name]['val_loss']
            print(f"Adjusted {name}'s learning rate to {new_lr:.2e}")

        if std_dev < threshold:
            print("Equilibrium reached among models.")
            break

    return std_devs

In [ ]:
std_devs = compete_until_equilibrium(models, dataloaders, criterion, device, num_epochs=50, threshold=0.01)

plt.plot(range(1, len(std_devs) + 1), std_devs, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Standard Deviation of Validation Losses')
plt.title('Standard Deviation vs Epochs')
plt.grid(True)
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
# Initialize models
model1 = CustomResNet50(4)
model2 = CustomResNet101(4)

# Load weights
model1.load_state_dict(torch.load('/kaggle/working/CustomResNet50.pth'))
model2.load_state_dict(torch.load('/kaggle/working/CustomResNet101.pth'))

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)
model2.to(device)

# Set models to evaluation mode
model1.eval()
model2.eval()

# Validation dataloader
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_dataset = datasets.ImageFolder('/kaggle/input/ham10000-data/HAM10000_DATA/test_dir', transform=val_transforms)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Function to get predictions and labels
def get_predictions_and_labels(model, dataloader, device):
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            all_outputs.append(probabilities)
            all_labels.append(labels)
    return torch.cat(all_outputs, dim=0), torch.cat(all_labels, dim=0)

# Load models, set device, dataloaders etc. from previous setup

# Get predictions and labels from both models
predictions1, labels = get_predictions_and_labels(model1, val_loader, device)
predictions2, _ = get_predictions_and_labels(model2, val_loader, device)

# Ensemble predictions (simple average of softmax outputs)
ensemble_predictions = (predictions1 + predictions2) / 2

# Determine final predicted classes
final_predictions = torch.argmax(ensemble_predictions, dim=1).cpu().numpy()
labels = labels.cpu().numpy()

# Calculate accuracy
accuracy = (final_predictions == labels).mean()
print(f"Ensemble accuracy on validation data: {accuracy:.4f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(labels, final_predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(labels, final_predictions)
print("Classification Report:")
print(class_report)

In [ ]:
# Validation dataloader
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dataset = datasets.ImageFolder('/kaggle/input/ham10000-data/HAM10000_DATA/test_dir', transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
data_transforms = {
    'test': transforms.Compose([
        transforms.Lambda(lambda x: x.convert('RGB')),
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
# Load image data
test_dir = '/kaggle/input/ham10000-data/HAM10000_DATA/test_dir'
image_datasets = {
    'test': ImageFolder(test_dir, transform=data_transforms['test'])
}

# Create DataLoader objects
dataloaders = {
    'test': DataLoader(image_datasets['test'], batch_size=4, shuffle=False)
    
}

In [ ]:
def evaluate_model_and_collect_outputs(model, dataloader, device):
    model.eval()
    total_outputs = []
    total_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            total_outputs.append(outputs.cpu())
            total_labels.append(labels.cpu())

    total_outputs = torch.cat(total_outputs, dim=0)
    total_labels = torch.cat(total_labels, dim=0)
    return total_outputs, total_labels

# Collect outputs and labels
outputs_resnet50, labels = evaluate_model_and_collect_outputs(models['CustomResNet50']['model'], dataloaders['test'], device)
outputs_resnet101, _ = evaluate_model_and_collect_outputs(models['CustomResNet101']['model'], dataloaders['test'], device)

In [ ]:
from sklearn.metrics import accuracy_score

# Compute accuracies for individual models
_, preds_resnet50 = torch.max(outputs_resnet50, 1)
_, preds_resnet101 = torch.max(outputs_resnet101, 1)
acc_resnet50 = accuracy_score(labels.numpy(), preds_resnet50.numpy())
acc_resnet101 = accuracy_score(labels.numpy(), preds_resnet101.numpy())

# Compute ensemble accuracy
ensemble_outputs = (outputs_resnet50 + outputs_resnet101) / 2
_, ensemble_preds = torch.max(ensemble_outputs, 1)
ensemble_acc = accuracy_score(labels.numpy(), ensemble_preds.numpy())

# Calculate Shapley values
phi_resnet50 = (acc_resnet50 + ensemble_acc - acc_resnet101) / 2
phi_resnet101 = (acc_resnet101 + ensemble_acc - acc_resnet50) / 2

In [ ]:
weighted_ensemble_outputs = (phi_resnet50 * outputs_resnet50 + phi_resnet101 * outputs_resnet101) / (phi_resnet50 + phi_resnet101)
_, weighted_ensemble_preds = torch.max(weighted_ensemble_outputs, 1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(labels.numpy(), weighted_ensemble_preds.numpy())
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion Matrix for Weighted Ensemble')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(labels.numpy(), weighted_ensemble_preds.numpy(), target_names=['akiec', 'bcc', 'bkl','df', 'mel', 'nv', 'vasc'])

print("Classification Report for Weighted Ensemble:")
print(report)

In [ ]:
!zip -r model_files.zip /kaggle/working/CustomResNet101.pth

In [ ]:
!zip -r model_files.zip /kaggle/working/CustomResNet50.pth

In [ ]:
import torch
softmax = torch.nn.Softmax(dim=1)
probabilities = softmax(weighted_ensemble_outputs).numpy()[:, 1]  # Probabilities for class 1

In [ ]:
print(labels.numpy().shape)

In [ ]:
!cd /kaggle/working
from IPython.display import FileLink
FileLink(r'model_files.zip')